### Headless components

Bezgłowe komponenty są to komponenty, które nie posiadają żadnego UI, lecz jedynie zapewniają pewną zdefiniowaną funkcjonalność.
Takie komponenty pozwalają na wielokrotne ich używanie i właśnie najlepiej sprawdzają się w aplikacjach, które będą wymagały powtarzalności funkcjonalności.

Przykładem będzie przycisk, który po naciśnięciu zmienia stan (może np ukrywać, lub pokazywać rozwijaną listę).

Zaczynamy od stworzenia katalogu, który będzie zawierał wszystkie komponenty, w tym przypadku katalog Toggle (w ../src).
W katalogu tworzymy najpierw Toggle.js.

In [ ]:
import React, { Children, useState } from 'react'
import { createContext } from 'react'

const ToggleContext = createContext() ## create context to keep and pass data

export default function Toggle({ children }) {  # export default function

  const [on, setOn] = useState(false) # defined state of the Toggle component

  function toggleOn() {   # function to toggle state of "on"
    setOn(prevOn => !prevOn)
  }

  return (
    <ToggleContext.Provider value={{on, toggleOn}}>  # providing Context to child components
      {children}
    </ToggleContext.Provider>
  )
}

export { ToggleContext } # export of ToggleContext

Następnie tworzymy kolejne sub-komponenty, które będą stanowiły całość komponentu Toggle.

In [ ]:
## ToggleButton.js
import React, { useContext } from 'react'
import { ToggleContext } from './Toggle' # import of ToggleContext to get data (in this case function to handle toggle click)

function ToggleButton({children}) {

  const {toggleOn} = useContext(ToggleContext) # destructured ToggleContext

  return (
    <button onClick={toggleOn}>{ children }</button> # we use toggleOn to change the state of "on" in Toggle.js
  )
}

export default ToggleButton

In [ ]:
# ToggleOn.js
import React, { useContext } from 'react'
import { ToggleContext } from './Toggle' # import of ToggleContext to get data (in this case function state "on")

function ToggleOn({children}) {

  const {on} = useContext(ToggleContext)

  return (
    on ? <div>{children}</div> : null # if "on" then render the component
  )
}

export default ToggleOn

In [ ]:
# ToggleOff.js - it's not necessary because usually we don't want to show anything when the toggle is off, but sometimes
# design requires to show some information when the toggle is off.
import React, { useContext } from 'react'
import { ToggleContext } from './Toggle' # import of ToggleContext to get data (in this case function state "on")

function ToggleOff({children}) {

  const {on} = useContext(ToggleContext)

  return (
    !on ? <div>{children}</div> : null # if "on" then render the component
  )
}

export default ToggleOff

Kiedy mamy już wszystkie komponenty, wówczas tworzymy w tym samym katalogu plik index.js, który będzie spinał wszystkie komponenty, aby nie trzeba było ich importować za każdym razem do innego komponentu.

In [ ]:
# index.js in Toggle folder

# import of parent Toggle component
import Toggle from "./Toggle";
#import of all other child components
import ToggleButton from "./ToggleButton"; 
import ToggleOff from "./ToggleOff";
import ToggleOn from "./ToggleOn";

Toggle.Button = ToggleButton # we declare Toggle.Button object as ToggleButton, and we do the same with other objects
Toggle.On = ToggleOn
Toggle.Off = ToggleOff

export default Toggle

Teraz, kiedy mamy przygotowany cały bezgłowy komponent, możemy użyć go w aplikacji.

In [ ]:
# App.js or other file/component we want to use it

# import of Toggle component (but from the index file where all components are pulled together)
import Toggle from './Toggle/index';

function App() {

  const items = ["item 1", "item 2" , "item"]

  return (
    <div className="App">
      <Toggle>
        <Toggle.Button> # this component will be responsbile for call of function toggle when clicked
          Press me
        </Toggle.Button>
        <Toggle.On> # this component will be rendered when the state in "on"
          It's on
        </Toggle.On>
        <Toggle.Off> # this component will be render when the state is "off"
          It's off
        </Toggle.Off>
      </Toggle>
    </div>
  );
}

Załóżmy teraz, że chcemy zastosować powyższy gotowy komponent do wyświetlenia rozwijanego menu. Menu składa się z przycisku, menu rozwijanego, które posiada pojedyncze elementy menu. Każdy z elementów przyjmuje właściwość children.

In [ ]:
# Menu.js
function Menu({ children }) {
  return (
      <div className='menu'>
        {children}
      </div>
  )
}

#MenuButton.js
export default function MenuButton( { children }) {
  return (
    <Toggle.Button>
      <Button>{children}</Button>
    </Toggle.Button>
  )
}

#MenuDropdown.js
function MenuDropdown( { children }) {
  return (
      <div className='menu-dropdown'>
        {children}
      </div>
  )
}

#MenuItem.js
export default function MenuItem({ children }) {
   return (
    <div className='menu-item'>
      {children}
    </div>
  )
}

#index.js - combining all above together
import Menu from "./Menu";
import MenuButton from "./MenuButton";
import MenuDropdown from "./MenuDropdown";
import MenuItem from "./MenuItem";

Menu.Button = MenuButton
Menu.Dropdown = MenuDropdown
Menu.Item = MenuItem

export default Menu

Teraz możemy umieścić Menu w komponencie Toggle, który będzie odpowiedzialny za funkcjonalność.

In [ ]:
function App() {
  return (
    <>
      <Toggle>
          <Menu>  # Menu wrapped in Toggle
            <Toggle.Button>
              <Menu.Button> # MenuButton wrapped in ToggleButton
                Menu
              </Menu.Button>
            </Toggle.Button>
            <Toggle.On> # When the state "on" is true MenuDropdown is wrapped in ToggleOn component
              <Menu.Dropdown>
                <Menu.Item>Home</Menu.Item>
                <Menu.Item>About</Menu.Item>
                <Menu.Item>Contact</Menu.Item>
                <Menu.Item>Blog</Menu.Item>
              </Menu.Dropdown>
            </Toggle.On>
            <Toggle.Off> # ToggleOff can be omitted if you don't want to render anything when the state "on" is set to false
            </Toggle.Off>
          </Menu>
      </Toggle>
    </>
  );
}

Powyższy kod jest jak najbardziej prawidłowy, ale jest dosyć mocno rozbudowany. Można go skrócić i sprawić, że będzie bardziej deskryptywny. 

Na początku rozszeramy komponent "opakowując" go w komponencie Toggle.

In [ ]:
#Menu.js
import React from 'react'
import Toggle from '../Toggle/index'  # import Toggle component

function Menu({ children }) {
  return (
    <Toggle> # wrap Menu component in Toggle componeent
      <div className='menu'>
        {children}
      </div>
    </Toggle>
  )
}

export default Menu

To samo robimy z MenuButton, ale tym razem opakowujemy go w ToggleButton.

In [ ]:
#MenuButton.js
import React from 'react'
import Button from '../Button/Button'
import Toggle from '../Toggle/index'
export default function MenuButton( { children }) {
  return (
    <Toggle.Button>
      <Button>{children}</Button>
    </Toggle.Button>
  )
}

I tak samo postępujemy z MenuDropdown, które chcemy wyrenderować, kiedy stan "on" będzie przyjmował wartość true.

In [ ]:
#MenuDropdown.js
import React from 'react'
import Toggle from '../Toggle/index'

export default function MenuDropdown( { children }) {
  return (
    <Toggle.On>
      <div className='menu-dropdown'>
        {children}
      </div>
    </Toggle.On>
  )
}

Dzięki temu, nasz kod w funkcji App() wygląda teraz o wiele czytelniej.

In [ ]:
function App() {
  return (
    <>
      # There is no Toggle elements in code anymore, code is more clear and descriptive.
      <Menu>
        <Menu.Button>
          Menu
        </Menu.Button>
        <Menu.Dropdown>
          <Menu.Item>Home</Menu.Item>
          <Menu.Item>About</Menu.Item>
          <Menu.Item>Contact</Menu.Item>
          <Menu.Item>Blog</Menu.Item>
        </Menu.Dropdown>
      </Menu>
    </>
  );
}